In [ ]:
!pip install --pre --upgrade azure-ai-ml

In [ ]:
!pip install opencv_contrib_python~=4.5.5 torch==1.8.1 torchvision==0.9.1 azure-storage-blob==12.8.1

In [ ]:
image_project = True
video_project = False
minimum_crop_size = 0
azure_automl_name = '<automl_name_of_your_job>'
azure_storage_connection_string = "<your_azure_storage_connection_string>"
azure_storage_container_input = "<container_name_for_raw_images>"
azure_storage_container_prefix="<prefix_for_raw_images>" # e.g. "" if none or "folder1/folder2/"
azure_storage_container_output = "<container_name_for_cropped_images>" # only lower case letters, nubers and dashes
local_directory_name = "<directory_name_for_aml_artifacts>" # e.g. "aml_artifacts"

# Optional - if using captured video
azure_sas_uri_for_video = "<your_azure_sas_uri_for_video>" # e.g. "https://<storage_account_name>.blob.core.windows.net/<container_name>/<video_name>?<sas_token>"


Import job information and artifacts from AML, including the ONNX file from the training artifacts.

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = ''   
    resource_group = ''  
    workspace_name = ''
    ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

In [ ]:
import mlflow
from mlflow.tracking.client import MlflowClient

# Had to add to documentation
mlflow_client = MlflowClient()

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# Specify the job name
job_name = azure_automl_name

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)
best_child_run_id = mlflow_parent_run.data.tags['automl_best_child_run_id']
# get the best child run
best_run = mlflow_client.get_run(best_child_run_id)

In [ ]:
import os

local_dir = local_directory_name
if not os.path.exists(local_dir):
    os.mkdir(local_dir)


In [ ]:
labels_file = mlflow_client.download_artifacts(
    best_run.info.run_id, 'train_artifacts/labels.json', local_dir
)

In [ ]:
onnx_model_path = mlflow_client.download_artifacts(
    best_run.info.run_id, 'train_artifacts/model.onnx', local_dir
)

In [ ]:
conda_file = mlflow_client.download_artifacts(
    best_run.info.run_id, 'outputs/conda_env_v_1_0_0.yml', local_dir
)

In [ ]:
from azure.ai.ml.entities import Environment

env = Environment(
    name="automl-images-env-onnx",
    description="environment for automl images ONNX batch model generation",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.1-cudnn8-ubuntu18.04",
    conda_file=conda_file,
)

Load model for inferencing

In [ ]:
import json
import onnxruntime

with open(labels_file) as f:
    classes = json.load(f)
print(classes)
try:
    session = onnxruntime.InferenceSession(onnx_model_path)
    print("ONNX model loaded...")
except Exception as e: 
    print("Error loading ONNX file: ", str(e))

In [ ]:
sess_input = session.get_inputs()
sess_output = session.get_outputs()
print(f"No. of inputs : {len(sess_input)}, No. of outputs : {len(sess_output)}")

for idx, input_ in enumerate(range(len(sess_input))):
    input_name = sess_input[input_].name
    input_shape = sess_input[input_].shape
    input_type = sess_input[input_].type
    print(f"{idx} Input name : { input_name }, Input shape : {input_shape}, \
    Input type  : {input_type}")  

for idx, output in enumerate(range(len(sess_output))):
    output_name = sess_output[output].name
    output_shape = sess_output[output].shape
    output_type = sess_output[output].type
    print(f" {idx} Output name : {output_name}, Output shape : {output_shape}, \
    Output type  : {output_type}")

In [ ]:
batch, channel, height_onnx, width_onnx = session.get_inputs()[0].shape
batch, channel, height_onnx, width_onnx

In [ ]:
from azure.storage.blob import BlobServiceClient, ContainerClient, BlobClient, StorageStreamDownloader

from azure.core.exceptions import ResourceExistsError

storage_client = BlobServiceClient.from_connection_string(azure_storage_connection_string)

try:
    input_container_client = storage_client.create_container(azure_storage_container_input)
        
except ResourceExistsError as e:
    print(e)
    input_container_client = storage_client.get_container_client(azure_storage_container_input)

try:
    output_container_client = storage_client.create_container(azure_storage_container_output)
        
except ResourceExistsError as e:
    print(e)
    output_container_client = storage_client.get_container_client(azure_storage_container_output)


In [ ]:
# import sys
# sys.path.append(r'/home/azureuser/cloudfiles/code/Users/<user name>/<path to notebook>')
from onnxruntime_yolov5 import initialize_yolov5

target_dim = 640
target_iou = .45
target_prob = .50

initialize_yolov5(onnx_model_path, labels_file, target_dim, target_prob, target_iou)

In [ ]:
def check_extension(filename):
    file_extensions = set(['png', 'jpg', 'jpeg', 'bmp', 'tiff', 'tif'])
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in file_extensions

In [ ]:
import os
import io
import cv2
import time
import json
import uuid
import numpy as np
from frame_preprocess import frame_resize
from onnxruntime_yolov5 import predict_yolov5
from datetime import datetime
import matplotlib.pyplot as plt
import cv2
from PIL import Image

frame_count = 0
crop_count = 0

if image_project:
    images = input_container_client.list_blobs(name_starts_with=azure_storage_container_prefix)
    for blob in images:
        image_name = blob.name
        if check_extension(image_name):
            frame_count += 1
            blob_client = input_container_client.get_blob_client(blob.name)
            blob_stream = blob_client.download_blob()
            blob_bytes = blob_stream.readall()
            frame = Image.open(io.BytesIO(blob_bytes))
            frame = np.array(frame)
            h, w = frame.shape[:2]
            print(f"Frame size: {w} x {h}")
            frame_optimized, ratio, pad_list = frame_resize(frame, target_dim, model = "yolov5")
            result = predict_yolov5(frame_optimized, pad_list)
            predictions = result['predictions'][0]

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # if result is not None:
                # print(json.dumps(result))

            if predictions is not None:
                detection_count = len(predictions)
                t_infer = result["inference_time"]
                # print(f"Inference Time: {t_infer}ms")
                # print(f"Detection Count: {detection_count}")

            now = datetime.now()
            created = now.isoformat()

            if detection_count > 0:
                for i in range(detection_count):
                    unique_id = str(uuid.uuid4())
                    crop_name = f"{created}-{frame_count}-{unique_id}.jpg"
                    bounding_box = predictions[i]['bbox']
                    tag_name = predictions[i]['labelName']
                    object_crop = frame[int(bounding_box["top"]/ratio[1]): int(bounding_box["height"]/ratio[1]), int(bounding_box["left"]/ratio[0]): int(bounding_box["width"]/ratio[0])]
                    
                    # increase crop size by 100 in each direction
                    # object_crop = frame[int(bounding_box["top"]/ratio[1]-100): int(bounding_box["height"]/ratio[1]+100), int(bounding_box["left"]/ratio[0]-100): int(bounding_box["width"]/ratio[0]+100)]
                    
                    object_h, object_w = object_crop.shape[:2]
                    print(f"Object width x height: {object_w} x {object_h}")

                    if minimum_crop_size > 0:
                        if object_h and object_w >= minimum_crop_size:
                            crop_count += 1
                            encoded_crop = cv2.imencode(".jpg", object_crop)[1].tobytes()
                            try:
                                blob_client = output_container_client.get_blob_client(f"{tag_name}/{crop_name}")
                                blob_client.upload_blob(encoded_crop, blob_type="BlockBlob")
                                print(f"Successfully uploaded {crop_name}")
                                # plt.imshow(object_crop)
                                # plt.show()
                            except Exception as e:
                                print(e)

                    else:
                        crop_count += 1
                        encoded_crop = cv2.imencode(".jpg", object_crop)[1].tobytes()
                        try:
                            blob_client = output_container_client.get_blob_client(f"{tag_name}/{crop_name}")
                            blob_client.upload_blob(encoded_crop, blob_type="BlockBlob")
                            print(f"Successfully uploaded {crop_name}")
                            # plt.imshow(object_crop)
                            # plt.show()
                        except Exception as e:
                            print(e)

elif video_project:
    frame_rate_count = 0
    camera_fps = 30
    inference_fps = 1
    vid_cap = cv2.VideoCapture(azure_sas_uri_for_video)
    while vid_cap.isOpened():
        _, frame = vid_cap.read()
        if _ == True:
            frame_count += 1
            frame_rate_count += 1
            if inference_fps > 0:
                if frame_rate_count == int(camera_fps/inference_fps):
                    frame_rate_count = 0
                    h, w = frame.shape[:2]
                    frame_optimized, ratio, pad_list = frame_resize(frame, target_dim, model = "yolov5")
                    result = predict_yolov5(frame_optimized, pad_list)
                    predictions = result['predictions'][0]
                    
                    # if result is not None:
                        # print(json.dumps(result))

                    if predictions is not None:
                        detection_count = len(predictions)
                        t_infer = result["inference_time"]
                        # print(f"Inference Time: {t_infer}ms")
                        # print(f"Detection Count: {detection_count}")

                    now = datetime.now()
                    created = now.isoformat()

                    if detection_count > 0:
                        for i in range(detection_count):
                            unique_id = str(uuid.uuid4())
                            crop_name = f"{created}-{frame_count}-{unique_id}.jpg"
                            bounding_box = predictions[i]['bbox']
                            tag_name = predictions[i]['labelName']
                            object_crop = frame[int(bounding_box["top"]/ratio[1]): int(bounding_box["height"]/ratio[1]), int(bounding_box["left"]/ratio[0]): int(bounding_box["width"]/ratio[0])]
                            
                            # increase crop size by 100 in each direction
                            # object_crop = frame[int(bounding_box["top"]/ratio[1]-100): int(bounding_box["height"]/ratio[1]+100), int(bounding_box["left"]/ratio[0]-100): int(bounding_box["width"]/ratio[0]+100)]
                            
                            object_h, object_w = object_crop.shape[:2]
                            print(f"Object width x height: {object_w} x {object_h}")

                            if minimum_crop_size > 0:
                                if object_h and object_w >= minimum_crop_size:
                                    crop_count += 1
                                    encoded_crop = cv2.imencode(".jpg", object_crop)[1].tobytes()
                                    try:
                                        blob_client = output_container_client.get_blob_client(f"{tag_name}/{crop_name}")
                                        blob_client.upload_blob(encoded_crop, blob_type="BlockBlob")
                                        print(f"Successfully uploaded {crop_name}")
                                        # plt.imshow(object_crop)
                                        # plt.show()
                                    except Exception as e:
                                        print(e)

                            else:
                                crop_count += 1
                                encoded_crop = cv2.imencode(".jpg", object_crop)[1].tobytes()
                                try:
                                    blob_client = output_container_client.get_blob_client(f"{tag_name}/{crop_name}")
                                    blob_client.upload_blob(encoded_crop, blob_type="BlockBlob")
                                    print(f"Successfully uploaded {crop_name}")
                                    # plt.imshow(object_crop)
                                    # plt.show()
                                except Exception as e:
                                    print(e)

else:
    print("Please select a project type")


: 